In [1]:
import pandas as pd
import numpy as np
import datetime
import os

In [2]:
path = "./data_air_quality_thursday/"
files_in_dir = os.listdir(path)

In [3]:
# filename = "&station=1025date_from=2016-01-01&time_from=1&date_to=2016-10-25&time_from=23.csv"

In [4]:
def format_file_string(name):
    if name[-67] == "d":
        return name[:-67]
    elif name[-68] == "d":
        return name[:-68]
    else:
        raise Exception(f"Input not in desired format. It was {name}") 

In [5]:
files_in_dir = os.listdir(path)
check_if_loaded = []
list_of_dfs = []

for i in files_in_dir:
    group_files = []
    if format_file_string(i) not in check_if_loaded:
        for j in files_in_dir:
            if format_file_string(i) == format_file_string(j):
                group_files.append(path+j)
        df = pd.read_csv(group_files[0],parse_dates=[0])
        
        for k in group_files[1:]:
            df2 = pd.read_csv(k,parse_dates=[0])
            df = pd.concat([df,df2])
        df = df.sort_values(by=["Date"])
        list_of_dfs.append(df)
        check_if_loaded.append(format_file_string(i))
     #   print(f"loaded {format_file_string(i)}")
        

In [6]:
### We get duplicates because of the summer/ wintertime ... also we have to transform to utc for entso-e 
# Daylight saving time 2016 in Germany began at 02:00 on Sunday, 27 March and ended at 03:00 on Sunday, 30 October
# Daylight saving time 2017 in Germany began at 02:00 on Sunday, 26 March and ended at 03:00 on Sunday, 29 October
# Daylight saving time 2018 in Germany began at 02:00 on Sunday, 25 March and ended at 03:00 on Sunday , 28 October
# Daylight saving time 2019 in Germany began at 02:00 on Sunday, 31 March and ended at 03:00 on Sunday, 27 October
# Daylight saving time 2020 in Germany began at 02:00 on Sunday, 29 March and ended at 03:00 on Sunday, 25 October
# Summer utc = berlin - 2, Winter utc = Berlin - 1 

In [7]:
time_switches = [
    "2011-01-01 00:00:00",
    "2016-03-27 02:00:01",
    "2016-10-30 03:00:00",
    "2017-03-26 02:00:01",
    "2017-10-29 03:00:00",
    "2018-03-25 02:00:01",
    "2018-10-28 03:00:00",
    "2019-03-31 02:00:01",
    "2019-10-27 03:00:00",
    "2020-03-29 02:00:01",
    "2020-10-25 03:00:00"]

In [8]:
# Next we transform our data to utc

In [9]:
for cnt,i in enumerate(list_of_dfs):
    df = list_of_dfs[cnt]
    df = df.reset_index()
    for j in range(len(time_switches)-1):
        mask = ((df.loc[:,'Date'] > time_switches[j]) & (df.loc[:,'Date'] < time_switches[j+1]))
        if j%2 == 0:
            df.loc[mask,"Date"] -= datetime.timedelta(hours=1)
        else:
            df.loc[mask,"Date"] -= datetime.timedelta(hours=2)
            mask2 = df.loc[:,'Date']  == time_switches[j+1]
            df.loc[mask2.idxmax(),"Date"] -= datetime.timedelta(hours=2)
            mask2 = df.loc[:,'Date']  == time_switches[j+1]
            df.loc[mask2.idxmax(),"Date"] -= datetime.timedelta(hours=1)
    df = df .sort_values(by=["Date"])
    df = df.drop(columns="index")
    list_of_dfs[cnt] = df.copy()

In [11]:
for i in list_of_dfs:
    if i.loc[:,"Date"].duplicated().any():
        print("not succesfull")

In [13]:
from functools import reduce

In [14]:
list_merged = []
for i in range(0,len(list_of_dfs)-10,10):
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'],
                                            how='outer'), list_of_dfs[i:i+10])
    list_merged.append(df_merged.copy())

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'],
                                            how='outer'), list_of_dfs[len(list_merged)*10:])
list_merged.append(df_merged.copy())

In [15]:
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'],
                                            how='outer'), list_merged)

In [16]:
df_merged = df_merged.sort_values(by=["Date"])
df_merged = df_merged.replace(["-"],np.nan)

In [17]:
columns_dropped = []

for i in df_merged:
    number_nan_rows = df_merged.loc[df_merged.loc[:,i].isnull(),i].shape[0]
    percentage_nan_rows = number_nan_rows/df_merged.shape[0]*100
    if (percentage_nan_rows > 25):
        columns_dropped.append([i,percentage_nan_rows])
        df_merged = df_merged.drop(columns=[i])

In [18]:
# df_merged.loc[:, df_merged.columns != "Date"] = df_merged.loc[:, df_merged.columns != "Date"].astype("float32")

In [19]:
df_merged = df_merged.sort_values(by=["Date"])

In [20]:
raise ValueError("notebook stop cell below for saving")

ValueError: notebook stop cell below for saving

In [21]:
df_merged.to_csv("air_quality.csv",index=False)